In [1]:
import os

# Change working directory to the parent directory
os.chdir("/Users/megan/Thesis/")
print("Current working directory:", os.getcwd())

Current working directory: /Users/megan/Thesis


In [1]:
import pandas as pd

# Load the data again after reset
file1_path = 'output/baseline_sarimax/final_metrics_summary.csv'
file2_path = 'output/sarimax_with_google_trends/final_metrics_summary.csv'

# Read both CSVs
final_metrics_model1 = pd.read_csv(file1_path)
final_metrics_model2 = pd.read_csv(file2_path)

# Extract MAPE values and compare them
mape_comparison = final_metrics_model1['MAPE'] < final_metrics_model2['MAPE']

# Count how many items perform better in model 1 and model 2
model1_better_count = mape_comparison.sum()
model2_better_count = len(mape_comparison) - model1_better_count

model1_better_count, model2_better_count


(np.int64(315), np.int64(485))

In [4]:
# Calculate MAPE for Model 1 and Model 2
model_1_mape = final_metrics_model1['MAPE']
model_2_mape = final_metrics_model2['MAPE']

# Calculate the absolute difference in MAPE for each item
mape_difference = model_1_mape - model_2_mape

# Calculate the average MAPE difference
average_mape_difference = mape_difference.mean()

# Calculate the percentage decrease in MAPE for Model 2
percentage_decrease = (average_mape_difference / model_1_mape.mean()) * 100

average_mape_difference, percentage_decrease

(np.float64(-21.339726731373457), np.float64(-69.03004626033781))

In [5]:
model_1_mape.mean()

np.float64(30.91367873475481)

In [8]:
import pandas as pd

# Load preprocessed
grouped_data = pd.read_csv('data/top_800_product_groups.csv')
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')
grouped_data.columns
data = grouped_data

In [30]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/baseline_sarimax/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# ✅ Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# ✅ Debug: Print column names if needed
print("Metrics Columns:", metrics_df.columns)
print("Grouped Data Columns:", grouped_data.columns)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Compute total days per product group
total_days = grouped_data.groupby('product_group')['date'].count()

# Calculate coverage percentage (non-zero transaction days / total days)
coverage_df = (transaction_days / total_days).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# ✅ Check if `product_group` exists in both before merging
if 'product_group' in metrics_df.columns and 'product_group' in coverage_df.columns:
    merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')
else:
    print("⚠️ 'product_group' column missing in one of the DataFrames!")
    print("Metrics DF columns:", metrics_df.columns)
    print("Coverage DF columns:", coverage_df.columns)
    exit()  # Stop execution if there's a problem

# Remove low-coverage product groups (<80% transactions)
filtered_df = merged_df[merged_df['coverage'] >= 0.8].copy()

# Drop coverage column after filtering
filtered_df.drop(columns=['coverage'], inplace=True)

# Save cleaned metrics
filtered_df.to_csv("output/baseline_sarimax/filtered_metrics_summary.csv", index=False)

# ✅ Recalculate Average Metrics After Removing Outliers
avg_metrics = {
    'MAE': filtered_df['MAE'].mean(),
    'RMSE': filtered_df['RMSE'].mean(),
    'MAPE': filtered_df['MAPE'].mean(),
    'R2': filtered_df['R2'].mean()
}

# Save new averages
pd.DataFrame([avg_metrics]).to_csv("output/baseline_sarimax/filtered_avg_metrics.csv", index=False)

print("\n✅ Process complete: Outliers removed based on <80% transaction coverage.")
print("\n📊 New Average Metrics:")
print(pd.DataFrame([avg_metrics]))


Metrics Columns: Index(['product_group', 'MAE', 'RMSE', 'MAPE', 'R2'], dtype='object')
Grouped Data Columns: Index(['date', 'product_group', 'transaction_count', 'avg_price',
       'unique_customers', 'std_price', 'unique_articles_sold', 'median_age',
       'club_member_ratio', 'fashion_news_subscribers',
       'first_purchase_days_ago', 'recent_purchase_days_ago',
       'sales_channel_2', 'most_common_age_bin_20-29',
       'most_common_age_bin_30-39', 'most_common_age_bin_40-49',
       'most_common_age_bin_50-59', 'most_common_age_bin_60+'],
      dtype='object')

✅ Process complete: Outliers removed based on <80% transaction coverage.

📊 New Average Metrics:
        MAE       RMSE       MAPE          R2
0  4.770862  10.983168  30.913679 -431.830377


In [31]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/baseline_sarimax_14_days/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# ✅ Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# ✅ Debug: Print column names if needed
print("Metrics Columns:", metrics_df.columns)
print("Grouped Data Columns:", grouped_data.columns)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Compute total days per product group
total_days = grouped_data.groupby('product_group')['date'].count()

# Calculate coverage percentage (non-zero transaction days / total days)
coverage_df = (transaction_days / total_days).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# ✅ Check if `product_group` exists in both before merging
if 'product_group' in metrics_df.columns and 'product_group' in coverage_df.columns:
    merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')
else:
    print("⚠️ 'product_group' column missing in one of the DataFrames!")
    print("Metrics DF columns:", metrics_df.columns)
    print("Coverage DF columns:", coverage_df.columns)
    exit()  # Stop execution if there's a problem

# Remove low-coverage product groups (<80% transactions)
filtered_df = merged_df[merged_df['coverage'] >= 0.8].copy()

# Drop coverage column after filtering
filtered_df.drop(columns=['coverage'], inplace=True)

# Save cleaned metrics
filtered_df.to_csv("output/baseline_sarimax/filtered_metrics_summary.csv", index=False)

# ✅ Recalculate Average Metrics After Removing Outliers
avg_metrics = {
    'MAE': filtered_df['MAE'].mean(),
    'RMSE': filtered_df['RMSE'].mean(),
    'MAPE': filtered_df['MAPE'].mean(),
    'R2': filtered_df['R2'].mean()
}

# Save new averages
pd.DataFrame([avg_metrics]).to_csv("output/baseline_sarimax/filtered_avg_metrics.csv", index=False)

print("\n✅ Process complete: Outliers removed based on <80% transaction coverage.")
print("\n📊 New Average Metrics:")
print(pd.DataFrame([avg_metrics]))


Metrics Columns: Index(['product_group', 'MAE', 'RMSE', 'MAPE', 'R2'], dtype='object')
Grouped Data Columns: Index(['date', 'product_group', 'transaction_count', 'avg_price',
       'unique_customers', 'std_price', 'unique_articles_sold', 'median_age',
       'club_member_ratio', 'fashion_news_subscribers',
       'first_purchase_days_ago', 'recent_purchase_days_ago',
       'sales_channel_2', 'most_common_age_bin_20-29',
       'most_common_age_bin_30-39', 'most_common_age_bin_40-49',
       'most_common_age_bin_50-59', 'most_common_age_bin_60+'],
      dtype='object')

✅ Process complete: Outliers removed based on <80% transaction coverage.

📊 New Average Metrics:
        MAE      RMSE       MAPE        R2
0  2.290747  2.977739  16.346171 -0.852919


In [41]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/baseline_sarimax/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Define the full 2-year period (inclusive)
full_period_start = pd.to_datetime("2018-09-20")
full_period_end = pd.to_datetime("2020-09-22")
total_days_in_range = (full_period_end - full_period_start).days + 1  # 734 days

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count the number of days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Use the fixed total days in range for coverage calculation
coverage_df = (transaction_days / total_days_in_range).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# Merge coverage info with metrics
merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')

# Identify Outliers (Based on Low Coverage <80% or High MAPE/R² Issues)
outliers_df = merged_df[(merged_df['coverage'] < 0.8) | (merged_df['MAPE'] > 100) | (merged_df['R2'] < 0)]

# Save the outliers to a CSV file
output_file_path = "output/baseline_sarimax/outliers_summary.csv"
outliers_df.to_csv(output_file_path, index=False)
outliers_df

,product_group,MAE,RMSE,MAPE,R2,coverage
8,Bikini top Beige All over pattern,13.047463,18.170007,119.972401,-0.420493,0.615804
16,Bikini top Dark Green All over pattern,4.658369,7.455192,30.474968,0.886069,0.634877
21,Bikini top Light Beige All over pattern,6.318970,27.223031,72.207053,0.862306,0.771117
22,Bikini top Light Blue All over pattern,5.499646,6.888212,200.716594,0.583783,0.780654
23,Bikini top Light Blue Solid,16.706606,20.244897,792.110492,-4.689503,0.606267
...,...,...,...,...,...,...
795,Bikini top Other Pink Other structure,7.122505,12.220479,32.246653,0.619691,0.373297
796,Swimwear bottom Other Pink Other structure,23.632806,31.617869,135.770396,-3.361680,0.371935
797,Bikini top Light Purple Other structure,611.879133,4167.335031,40.050463,-341072.719578,0.299728
798,Dress White Other structure,2.239700,2.722363,19.650136,0.926608,0.254768


In [43]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/baseline_sarimax_14_days/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Define the full 2-year period (inclusive)
full_period_start = pd.to_datetime("2018-09-20")
full_period_end = pd.to_datetime("2020-09-22")
total_days_in_range = (full_period_end - full_period_start).days + 1  # 734 days

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count the number of days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Use the fixed total days in range for coverage calculation
coverage_df = (transaction_days / total_days_in_range).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# Merge coverage info with metrics
merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')

# Identify Outliers (Based on Low Coverage <80% or High MAPE/R² Issues)
outliers_df = merged_df[(merged_df['coverage'] < 0.8) | (merged_df['MAPE'] > 100) | (merged_df['R2'] < 0)]

# Save the outliers to a CSV file
output_file_path = "output/baseline_sarimax_14_days/outliers_summary.csv"
outliers_df.to_csv(output_file_path, index=False)
outliers_df

,product_group,MAE,RMSE,MAPE,R2,coverage
8,Bikini top Beige All over pattern,3.009779,3.974976,44.346719,0.417361,0.615804
16,Bikini top Dark Green All over pattern,2.530283,2.894461,14.222212,0.946437,0.634877
21,Bikini top Light Beige All over pattern,3.133449,4.014688,40.969789,-0.289346,0.771117
22,Bikini top Light Blue All over pattern,1.865285,2.635603,16.590180,0.682210,0.780654
23,Bikini top Light Blue Solid,4.357989,4.707166,35.675518,0.745122,0.606267
...,...,...,...,...,...,...
795,Bikini top Other Pink Other structure,2.975118,3.704403,10.565924,0.895663,0.373297
796,Swimwear bottom Other Pink Other structure,2.996066,4.322782,9.250471,0.922707,0.371935
797,Bikini top Light Purple Other structure,4.768365,6.174249,20.775591,0.645914,0.299728
798,Dress White Other structure,2.235732,2.711287,15.083347,0.876597,0.254768


In [44]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/sarimax_with_google_trends/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Define the full 2-year period (inclusive)
full_period_start = pd.to_datetime("2018-09-20")
full_period_end = pd.to_datetime("2020-09-22")
total_days_in_range = (full_period_end - full_period_start).days + 1  # 734 days

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count the number of days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Use the fixed total days in range for coverage calculation
coverage_df = (transaction_days / total_days_in_range).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# Merge coverage info with metrics
merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')

# Identify Outliers (Based on Low Coverage <80% or High MAPE/R² Issues)
outliers_df = merged_df[(merged_df['coverage'] < 0.8) | (merged_df['MAPE'] > 100) | (merged_df['R2'] < 0)]

# Save the outliers to a CSV file
output_file_path = "output/sarimax_with_google_trends/outliers_summary.csv"
outliers_df.to_csv(output_file_path, index=False)
outliers_df

,product_group,MAE,RMSE,MAPE,R2,coverage
8,Bikini top Beige All over pattern,2.174139,3.083534,46.882761,0.855297,0.615804
16,Bikini top Dark Green All over pattern,9.244047,17.197817,44.302012,0.666400,0.634877
21,Bikini top Light Beige All over pattern,5.639414,24.709763,68.094480,0.860250,0.771117
22,Bikini top Light Blue All over pattern,5.291732,6.374338,191.182234,0.654124,0.780654
23,Bikini top Light Blue Solid,18.923580,22.443696,918.723791,-6.156361,0.606267
...,...,...,...,...,...,...
795,Bikini top Other Pink Other structure,6.314757,9.308901,30.536757,0.811067,0.373297
796,Swimwear bottom Other Pink Other structure,2371.502677,3185.511714,13399.786448,-56220.145510,0.371935
797,Bikini top Light Purple Other structure,8.913606,35.354849,42.319914,-17.431904,0.299728
798,Dress White Other structure,4.008262,4.492692,25.174260,0.855021,0.254768


In [45]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/sarimax_with_google_trends_14_days/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Define the full 2-year period (inclusive)
full_period_start = pd.to_datetime("2018-09-20")
full_period_end = pd.to_datetime("2020-09-22")
total_days_in_range = (full_period_end - full_period_start).days + 1  # 734 days

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count the number of days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Use the fixed total days in range for coverage calculation
coverage_df = (transaction_days / total_days_in_range).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# Merge coverage info with metrics
merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')

# Identify Outliers (Based on Low Coverage <80% or High MAPE/R² Issues)
outliers_df = merged_df[(merged_df['coverage'] < 0.8) | (merged_df['MAPE'] > 100) | (merged_df['R2'] < 0)]

# Save the outliers to a CSV file
output_file_path = "output/sarimax_with_google_trends_14_days/outliers_summary.csv"
outliers_df.to_csv(output_file_path, index=False)
outliers_df

,product_group,MAE,RMSE,MAPE,R2,coverage
8,Bikini top Beige All over pattern,2.182917,2.844289,19.202573,0.810273,0.615804
16,Bikini top Dark Green All over pattern,2.911611,3.263871,17.966805,0.928621,0.634877
21,Bikini top Light Beige All over pattern,3.067308,3.806347,28.870400,0.318340,0.771117
22,Bikini top Light Blue All over pattern,2.129347,2.842672,18.215975,0.659966,0.780654
23,Bikini top Light Blue Solid,4.644265,5.041362,37.758587,0.676686,0.606267
...,...,...,...,...,...,...
795,Bikini top Other Pink Other structure,2.984598,3.673566,9.215783,0.912122,0.373297
796,Swimwear bottom Other Pink Other structure,3.124763,4.399976,9.408233,0.930636,0.371935
797,Bikini top Light Purple Other structure,4.220636,5.525455,16.365569,0.736177,0.299728
798,Dress White Other structure,3.124570,3.504533,18.778477,0.821645,0.254768


In [34]:
import pandas as pd

# Load Metrics (Performance Results)
metrics_path = "output/sarimax_with_google_trends/final_metrics_summary.csv"
metrics_df = pd.read_csv(metrics_path)

# Load Transaction Data (To Compute % Days with Transactions)
data_path = "data/top_800_product_groups.csv"
grouped_data = pd.read_csv(data_path)

# ✅ Ensure Column Names Match (Rename for Consistency)
metrics_df.rename(columns={"Unnamed: 0": "product_group"}, inplace=True)
grouped_data.rename(columns={"Product Group": "product_group"}, inplace=True)

# ✅ Debug: Print column names if needed
print("Metrics Columns:", metrics_df.columns)
print("Grouped Data Columns:", grouped_data.columns)

# Ensure `date` is in datetime format
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Count days with transactions per product group
transaction_days = grouped_data.groupby('product_group')['transaction_count'].apply(lambda x: (x > 0).sum())

# Compute total days per product group
total_days = grouped_data.groupby('product_group')['date'].count()

# Calculate coverage percentage (non-zero transaction days / total days)
coverage_df = (transaction_days / total_days).reset_index()
coverage_df.columns = ['product_group', 'coverage']

# ✅ Check if `product_group` exists in both before merging
if 'product_group' in metrics_df.columns and 'product_group' in coverage_df.columns:
    merged_df = metrics_df.merge(coverage_df, on='product_group', how='left')
else:
    print("⚠️ 'product_group' column missing in one of the DataFrames!")
    print("Metrics DF columns:", metrics_df.columns)
    print("Coverage DF columns:", coverage_df.columns)
    exit()  # Stop execution if there's a problem

# Remove low-coverage product groups (<80% transactions)
filtered_df = merged_df[merged_df['coverage'] >= 0.8].copy()

# Drop coverage column after filtering
filtered_df.drop(columns=['coverage'], inplace=True)

# Save cleaned metrics
filtered_df.to_csv("output/baseline_sarimax/filtered_metrics_summary.csv", index=False)

# ✅ Recalculate Average Metrics After Removing Outliers
avg_metrics = {
    'MAE': filtered_df['MAE'].mean(),
    'RMSE': filtered_df['RMSE'].mean(),
    'MAPE': filtered_df['MAPE'].mean(),
    'R2': filtered_df['R2'].mean()
}

# Save new averages
pd.DataFrame([avg_metrics]).to_csv("output/baseline_sarimax/filtered_avg_metrics.csv", index=False)

print("\n✅ Process complete: Outliers removed based on <80% transaction coverage.")
print("\n📊 New Average Metrics:")
print(pd.DataFrame([avg_metrics]))


Metrics Columns: Index(['product_group', 'MAE', 'RMSE', 'MAPE', 'R2'], dtype='object')
Grouped Data Columns: Index(['date', 'product_group', 'transaction_count', 'avg_price',
       'unique_customers', 'std_price', 'unique_articles_sold', 'median_age',
       'club_member_ratio', 'fashion_news_subscribers',
       'first_purchase_days_ago', 'recent_purchase_days_ago',
       'sales_channel_2', 'most_common_age_bin_20-29',
       'most_common_age_bin_30-39', 'most_common_age_bin_40-49',
       'most_common_age_bin_50-59', 'most_common_age_bin_60+'],
      dtype='object')

✅ Process complete: Outliers removed based on <80% transaction coverage.

📊 New Average Metrics:
        MAE       RMSE       MAPE         R2
0  8.694712  13.045464  52.253405 -117.65704


In [10]:
subset_df = data[data['product_group'] == 'Pyjama bottom Black Solid'].copy()

# Print or display the subset
print("Rows for 'Pyjama bottom Black Solid':")
print(subset_df)

print("Max transaction_count:", subset_df['transaction_count'].max())
print("Min transaction_count:", subset_df['transaction_count'].min())
print("Number of rows:", len(subset_df))


Rows for 'Pyjama bottom Black Solid':
             date              product_group  transaction_count  avg_price  \
286    2018-09-20  Pyjama bottom Black Solid                 18   0.016603   
991    2018-09-21  Pyjama bottom Black Solid                 19   0.016843   
1668   2018-09-22  Pyjama bottom Black Solid                  8   0.016932   
2324   2018-09-23  Pyjama bottom Black Solid                 29   0.016932   
3023   2018-09-24  Pyjama bottom Black Solid                 44   0.016913   
...           ...                        ...                ...        ...   
531727 2020-09-18  Pyjama bottom Black Solid                 20   0.016568   
532423 2020-09-19  Pyjama bottom Black Solid                 21   0.016852   
533117 2020-09-20  Pyjama bottom Black Solid                 18   0.016374   
533814 2020-09-21  Pyjama bottom Black Solid                 19   0.016637   
534503 2020-09-22  Pyjama bottom Black Solid                 14   0.016827   

        unique_customers 

In [21]:
import pandas as pd

# Filter for the product group of interest
product_group = "Underwear bottom Dark Red Solid"
df_pg = data[data['product_group'] == product_group].copy()

# Ensure 'date' is datetime and sort
df_pg['date'] = pd.to_datetime(df_pg['date'])
df_pg = df_pg.set_index('date').asfreq('D').fillna(0)  # Fill missing days with 0

# Summary statistics
summary_stats = df_pg['transaction_count'].describe()

# Count days with zero sales
zero_sales_days = (df_pg['transaction_count'] == 0).sum()
total_days = df_pg.shape[0]
zero_sales_pct = (zero_sales_days / total_days) * 100

print(f"Product Group: {product_group}")
print(f"Total Days: {total_days}")
print(f"Days with Zero Sales: {zero_sales_days} ({zero_sales_pct:.2f}%)")
print(summary_stats)


Product Group: Underwear bottom Dark Red Solid
Total Days: 734
Days with Zero Sales: 16 (2.18%)
count     734.000000
mean       88.882834
std       109.761393
min         0.000000
25%        14.000000
50%        55.000000
75%       136.750000
max      1444.000000
Name: transaction_count, dtype: float64


In [19]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from modules.utils import process_name

output_dir = "output/baseline_sarimax_one_item_dropped_cols"
os.makedirs(output_dir, exist_ok=True)

# We'll still run only on this item of interest
item_of_interest = "Bikini top Blue Stripe"
sanitized_item_name = process_name(item_of_interest)  # Convert to safe filename

df_item = grouped_data[grouped_data['product_group'] == item_of_interest].copy()

# Convert date to datetime index
df_item['date'] = pd.to_datetime(df_item['date'])
df_item = df_item.set_index('date').asfreq('D').fillna(0)

# Define endog
endog = df_item['transaction_count']

# List out exogenous columns, dropping certain columns
drop_cols = ['transaction_count', 'product_group', 'std_price', 'club_member_ratio']
exog_cols = [c for c in df_item.columns if c not in drop_cols]
exog = df_item[exog_cols] if exog_cols else None

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=4)
if len(df_item) < (tscv.n_splits + 1):
    print(f"Not enough data points ({len(df_item)}) for 4 splits.")
    exit()

fold_metrics = []
fig, axes = plt.subplots(tscv.n_splits, 2, figsize=(20, 8 * tscv.n_splits))
fig.suptitle(f"{item_of_interest}: Drop 'std_price' & 'club_member_ratio'", fontsize=16, y=0.95)

for fold, (train_index, test_index) in enumerate(tscv.split(df_item)):
    train_endog = endog.iloc[train_index]
    test_endog = endog.iloc[test_index]
    train_exog = exog.iloc[train_index] if exog is not None else None
    test_exog = exog.iloc[test_index] if exog is not None else None

    model = SARIMAX(
        train_endog,
        exog=train_exog,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 7)
    )
    results = model.fit(disp=False, maxiter=500)

    print(f"\n=== Fold {fold+1} - Model Summary (Dropping std_price & club_member_ratio) ===")
    print(results.summary())

    pred = results.get_prediction(
        start=test_endog.index[0],
        end=test_endog.index[-1],
        exog=test_exog
    )
    pred_mean = np.maximum(pred.predicted_mean, 0)

    mae = mean_absolute_error(test_endog, pred_mean)
    rmse = np.sqrt(mean_squared_error(test_endog, pred_mean))
    # Avoid dividing by zero
    mape = np.mean(np.abs((test_endog - pred_mean) / test_endog.replace(0, np.nan))) * 100
    r2 = r2_score(test_endog, pred_mean)

    fold_metrics.append({
        "Fold": fold + 1,
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "R2": r2
    })

    # Plots
    axes[fold, 0].plot(test_endog.index, test_endog, label="Actual", alpha=0.7)
    axes[fold, 0].plot(test_endog.index, pred_mean, label="Predicted", alpha=0.7, linestyle='--')
    axes[fold, 0].set_title(f"Fold {fold+1}: Predicted vs Actual", fontsize=12)
    axes[fold, 0].legend()

    residuals = test_endog - pred_mean
    axes[fold, 1].plot(test_endog.index, residuals, label="Residuals")
    axes[fold, 1].axhline(0, linestyle="--", alpha=0.7)
    axes[fold, 1].set_title(f"Fold {fold+1}: Residuals", fontsize=12)
    axes[fold, 1].legend()

plt.tight_layout(rect=[0, 0, 1, 0.95])

# Use 'sanitized_item_name' in filenames
plot_path = os.path.join(output_dir, f"{sanitized_item_name}_drop_cols_cv.png")
plt.savefig(plot_path)
plt.close(fig)

metrics_df = pd.DataFrame(fold_metrics)
metrics_df_path = os.path.join(output_dir, f"{sanitized_item_name}_drop_cols_fold_metrics.csv")
metrics_df.to_csv(metrics_df_path, index=False)

avg_metrics = {
    'MAE': metrics_df['MAE'].mean(),
    'RMSE': metrics_df['RMSE'].mean(),
    'MAPE': metrics_df['MAPE'].mean(),
    'R2': metrics_df['R2'].mean()
}
avg_metrics_df = pd.DataFrame([avg_metrics])
avg_metrics_df_path = os.path.join(output_dir, f"{sanitized_item_name}_drop_cols_avg_metrics.csv")
avg_metrics_df.to_csv(avg_metrics_df_path, index=False)

print("\n===== Done! =====")
print(f"Fold metrics saved to: {metrics_df_path}")
print("Average metrics:\n", avg_metrics_df)
print(f"Plots saved to: {plot_path}")


=== Fold 1 - Model Summary (Dropping std_price & club_member_ratio) ===
                                     SARIMAX Results                                     
Dep. Variable:                 transaction_count   No. Observations:                  121
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood                -341.002
Date:                           Thu, 06 Feb 2025   AIC                            718.003
Time:                                   16:46:45   BIC                            767.096
Sample:                               02-12-2019   HQIC                           737.925
                                    - 06-12-2019                                         
Covariance Type:                             opg                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
avg_price          

In [ ]:
import pandas as pd

# Define the full date range
full_period_start = pd.to_datetime("2018-09-20")
full_period_end = pd.to_datetime("2020-09-22")

# Calculate total days in that period (inclusive)
total_days_in_range = (full_period_end - full_period_start).days + 1

for pg in outlier_list:
    # Filter to just rows for this product group
    df_pg = data[data['product_group'] == pg].copy()
    
    # Ensure 'date' is a datetime
    df_pg['date'] = pd.to_datetime(df_pg['date'])
    
    # Further restrict to the full date range in question
    df_pg = df_pg[(df_pg['date'] >= full_period_start) & (df_pg['date'] <= full_period_end)]
    
    # How many unique days are present in the dataset for this product group?
    unique_days_recorded = df_pg['date'].nunique()
    
    # Days with no data are effectively "missing"
    missing_days = total_days_in_range - unique_days_recorded
    
    coverage_pct = (unique_days_recorded / total_days_in_range) * 100
    
    print(f"Product Group: {pg}")
    print(f"  Total days in [2018-09-20 .. 2020-09-22]:  {total_days_in_range}")
    print(f"  Days recorded in dataset:                 {unique_days_recorded}")
    print(f"  Missing days (no rows recorded):          {missing_days}")
    print(f"  Coverage:                                  {coverage_pct:.2f}%\n")


Product Group: Bikini top Beige All over pattern
  Total days in [2018-09-20 .. 2020-09-22]:  734
  Days recorded in dataset:                 452
  Missing days (no rows recorded):          282
  Coverage:                                  61.58%

Product Group: Bikini top Light Blue All over pattern
  Total days in [2018-09-20 .. 2020-09-22]:  734
  Days recorded in dataset:                 573
  Missing days (no rows recorded):          161
  Coverage:                                  78.07%

Product Group: Bikini top Light Blue Solid
  Total days in [2018-09-20 .. 2020-09-22]:  734
  Days recorded in dataset:                 445
  Missing days (no rows recorded):          289
  Coverage:                                  60.63%

Product Group: Blazer Black Stripe
  Total days in [2018-09-20 .. 2020-09-22]:  734
  Days recorded in dataset:                 590
  Missing days (no rows recorded):          144
  Coverage:                                  80.38%

Product Group: Blouse Dark 

In [3]:
import os
import glob
import pandas as pd

# Folder containing subdirectories for each item
transfer_dir = "final_version/output/1_day/transfer"

# Use glob to find all avg_metrics.csv files under transfer/*
csv_files = glob.glob(os.path.join(transfer_dir, "*", "avg_metrics.csv"))

# List to store DataFrames of metrics
metrics_list = []

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        metrics_list.append(df)
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

if metrics_list:
    # Concatenate all metrics into one DataFrame
    all_metrics_df = pd.concat(metrics_list, ignore_index=True)
    # Compute the mean of each column and round to 4 decimals
    avg_metrics = all_metrics_df.mean().round(4)
    
    print("Average metrics over all items:")
    print(avg_metrics)
    
    # Optionally, save the results to a CSV file
    output_file = os.path.join(transfer_dir, "avg_metrics_all.csv")
    avg_metrics.to_csv(output_file, header=True)
else:
    print("No avg_metrics.csv files found in the transfer directory.")


Average metrics over all items:
Fold     2.5000
MAE      0.0525
RMSE     0.0837
MAPE    31.6770
R2       0.0766
dtype: float64
